In [1]:
#reading the dataframes and plot 
# RSRP= Reference Signal Receive Power
# RSRQ = Reference Signal Receive Quality
# RSSI = Received Signal Strength Indicator

In [2]:
import pandas as pd
import numpy as np
import os, glob
import matplotlib.pyplot as plt

In [4]:
path = "/Users/kpangalu/Downloads/RLF_relateddata/test/"
all_files = glob.glob(os.path.join(path, "cell-*.csv"))
print(len(all_files))

1


In [5]:
#this routine for reading dataframe and assigning the column names
prgname = 'rlf_rsrp_fns_mulplt.ipynb'

def readcsv(csvfile):
    global bgrnd, justfname,prgname
    #collect the folder name
    ssplt = csvfile.split("/")[-1]
    ssplt1 = ssplt.split('\\')[0]
    bgrnd = ssplt1.split('_')[-1]
    #print(bgrnd)
    
    #picking the filename
    sindx = [i for i, ltr in enumerate (csvfile) if ltr =='/']
    justfname = csvfile[max(sindx)+1:]
    dirname = csvfile[:max(sindx)]
    print('filename: ',justfname)
    csvfile1 = dirname + '/' + justfname
    #print('function:',csvfile)
    
    #reading the dataframe, changing column names
    df = pd.read_csv(csvfile, skiprows=1,low_memory=True, header=None)
    numcols = len(df.columns)
  
    if(numcols == 10):
        df.rename(columns={0:"Level", 1:"Unk", 2:"Poll", 3:"numb", 4:"NotR", 5:"RSSI", 6:"RSRP", 7:"RSRQ", 8:"TF", 9:"cellIDnum"}, inplace = True)
    if(numcols == 9):
        df.rename(columns={0:"Level", 1:"Poll", 2:"numb", 3:"NotR", 4:"RSSI", 5:"RSRP", 6:"RSRQ", 7:"TF", 8:"cellIDnum"}, inplace = True)

    #drop one column from the dataframe
    df.drop(['cellIDnum'], axis=1, inplace=True)
    
    #picking the every 50th value from the dataframe
    df1 = df[df.reset_index().index % 50 == 0]
    display(df1)
    df1.loc[df1['RSRP'] > 1.0, 'RSRP'] = 1
    df1.loc[df1['RSRP'] < -200.0, 'RSRP'] = -200.0
    
    return bgrnd, justfname, df, df1
    

In [9]:
#finding each variable indices in the data frame
search = [' ENTER_STATION',' EXIT_TRAIN', ' TRAIN_START', ' EXIT_STATION',' TRAIN_STOP',' ENTER_TRAIN']
nstrgs = len(search)

def str_strt_endpts(dframe):
    for k in range(len(search)):
        stm = search[k]
        #pick indeices corresponding strings
        data = df1.loc[df1.isin([stm]).any(axis=1)].index.tolist()
        print('data length :',len(data))
        if(len(data) < 1):
            stend= [0, 0]
        if(len(data) > 1):
            #find the indices >1 integers from the differences
            diff_lst = [data[i+1] - data[i] for i in range(len(data)-1)]
            diff_lst = list(map(lambda x: x//50, diff_lst))  #step = 50
            #print(diff_lst)
            sell = [i for i,v in enumerate(diff_lst) if v>1]
            #display('sell: ',sell)
        
            #preparing the actual values for plotting purpose
            nms = len(sell)
            if((len(sell) <= 0) or (len(data) > 0)):
                stend = [data[0], data[-1]]
        
            if (len(sell) > 0):
                stend = []
                for i in range(len(sell)):
                    if(i == 0): 
                        dum = data[0]+1
                        stend.append(dum)
                    pnt = sell[i]-1
                    dum = data[pnt]+2
                    stend.append(dum)
                    pnt = sell[i]+1
                    dum = data[pnt]+2
                    stend.append(dum)
        
                dum = data[-1]
                stend.append(dum)
        print(stm,stend)
        if(k == 0): enterstn = stend 
        if(k == 1): exttrn = stend
        if(k == 2): trnstrt = stend
        if(k == 3): extstn = stend
        if(k == 4): trnstp = stend
        if(k == 5): entertrn = stend  
    return enterstn,exttrn,trnstrt,extstn,trnstp,entertrn

In [10]:
#plot function using dataframe
kk = 0
def dataframe_plot(data, varib):
    global kk
    datan = data.dropna(subset=[varib])
    #display(data)
    ax1 = df1.plot(y=varib, color='black', figsize=(14,8),fontsize=18, legend=None)
    ax1.set_ylabel(varib, fontsize=20)
    
    #ax1.text(10,df1.RSRP.min()-1,justfname,fontsize=10)
    ax1.text(2,df1.RSRP.min()-6,prgname,fontsize=9)
    
    xmn, xmx, ymn, ymx = ax1.axis()
    # print(ymn, ymx)
    #vertical line (vline)
    for jj in range(len(search)):
        if(jj == 0): 
            stend = enterstn
            clr = 'blue'
        if(jj == 1): 
            stend = exttrn
            clr = 'green'
        if(jj == 2): 
            stend = trnstrt
            clr = 'red'
        if(jj == 3): 
            stend = extstn
            clr = 'magenta'
        if(jj == 4): 
            stend = trnstp
            clr = 'cyan'
        if(jj == 5): 
            stend = entertrn
            clr = 'yellow'
        stnnme = search[0]
        for ii in stend:
            plt.vlines(x=ii, ymin=ymn, ymax=ymx, linewidth=1, color=clr, linestyle = 'dashed')
        nme = search[jj]
        # Removing duplicates and nan from the list
        datt = (datan[varib]).tolist()
        datt1 = list(set(datt))
        datt1.sort()
        #picking second max value from the list of values(datt1[-2])
        spcefac = (datt1[-2] - datt1[0])/15
        ax1.text(10,ymx-spcefac*jj,nme,fontsize=14,color=clr)

    kk += 1 
    #outname = bgrnd + '_' + varib +'_'+str(kk)+'.pdf'
    outname = justfname +'_rsrp'+str(kk)+'.pdf'
    #outfle = '/Users/kpangalu/Downloads/RLF_relateddata/cell_data_plots_downscale50/' + outname
    outfle = '/Users/kpangalu/Downloads/' + outname
    plt.savefig(outfle, bbox_inches = "tight")       
    print(search[0])

In [11]:
for cname in all_files:
    print(cname)
    #reading the dataframe function
    sindx = [i for i, ltr in enumerate (cname) if ltr =='\\']
    justfname = cname[max(sindx)+1:]
    dirname = cname[:max(sindx)]
    #print(justfname, dirname)
    csvfile1 = dirname + '/' + justfname
    #print('csvfile: ',csvfile1)
    
    #reading the dataframe function
    fldrnme, cfname, df, df1 = readcsv(csvfile1)
     
    print(fldrnme,cfname)
    #display(df1.head())
    
    #display dataframe details
    print("Totoal number observations: ",len(df1.index))
    print("RSRP MIN and Max: ",df1.RSRP.min(),df1.RSRP.max())
    print("RSRQ MIN and Max: ",df1.RSRQ.min(),df1.RSRQ.max())
    print("RSSI MIN and Max: ",df1.RSSI.min(),df1.RSSI.max())
    
    #display(df1.index)
    #calc. the start, end indices from dataframe
    #search = [' ENTER_STATION',' EXIT_TRAIN', ' TRAIN_START', ' EXIT_STATION',' TRAIN_STOP',' ENTER_TRAIN']
    #nstrgs = len(search)
    
    #calc. the start, end indices from dataframe
    enterstn,exttrn,trnstrt,extstn,trnstp,entertrn = str_strt_endpts(df1)
    
    #plot for one variable profile and vertical lines are indices values of above function
    dataframe_plot(df1,'RSRP')     #select variable

/Users/kpangalu/Downloads/RLF_relateddata/test\cell-null-unknown-2023-04-13-23-44-05-370000000-0413MTP_Lingzhi.csv
filename:  cell-null-unknown-2023-04-13-23-44-05-370000000-0413MTP_Lingzhi.csv


Level           Unk     Poll           numb NotR  RSSI  RSRP  \
0       TRAIN_START  UNDER_GROUND  CI_POLL  1681401255637  LTE -51.0 -67.0   
50      TRAIN_START  UNDER_GROUND  CI_POLL  1681401257001  LTE -51.0 -67.0   
100     TRAIN_START  UNDER_GROUND  CI_POLL  1681401258282  LTE -51.0 -73.0   
150     TRAIN_START  UNDER_GROUND  CI_POLL  1681401259614  LTE -51.0 -73.0   
200     TRAIN_START  UNDER_GROUND  CI_POLL  1681401260903   NR   2.0 -81.0   
...             ...           ...      ...            ...  ...   ...   ...   
43450   TRAIN_START  UNDER_GROUND  CI_POLL  1681402367224  LTE -55.0 -76.0   
43500   TRAIN_START  UNDER_GROUND  CI_POLL  1681402368511  LTE -57.0 -77.0   
43550   TRAIN_START  UNDER_GROUND  CI_POLL  1681402369857  LTE -67.0 -90.0   
43600   TRAIN_START  UNDER_GROUND  CI_POLL  1681402371125  LTE -67.0 -90.0   
43650   TRAIN_START  UNDER_GROUND  CI_POLL  1681402372406  LTE -73.0 -96.0   

       RSRQ     TF  
0      -5.0  False  
50     -5.0  False  
100    -7.0  False  
150    -7.0  False  
200   -11.0  False  
...     ...    ...  
43450  -3.0  False  
43500  -3.0  False  
43550  -3.0  False  
43600  -3.0  False  
43650  -3.0  False  

[874 rows x 9 columns]

Lingzhi.csv cell-null-unknown-2023-04-13-23-44-05-370000000-0413MTP_Lingzhi.csv
Totoal number observations:  874
RSRP MIN and Max:  -140.0 1.0
RSRQ MIN and Max:  -38.0 -3.0
RSSI MIN and Max:  -73.0 4.0
data length : 0
 ENTER_STATION [0, 0]
data length : 367
 EXIT_TRAIN [22650, 40950]
data length : 329
 TRAIN_START [1, 1102, 2852, 6752, 8252, 12552, 14152, 16952, 18552, 21252, 42552, 43650]
data length : 0
 EXIT_STATION [0, 0]
data length : 147
 TRAIN_STOP [1201, 2752, 6852, 8152, 12652, 14052, 17052, 18452, 21352, 22600]
data length : 31
 ENTER_TRAIN [41000, 42500]


C:\Users\kpangalu\Anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


 ENTER_STATION
